<a href="https://colab.research.google.com/github/dmalch/genealogy-research/blob/main/GenealogyResearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Шаг 1

Открыть таблицы с индексированными данными из метрических книг.

In [7]:
import pandas as pd
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# Open the spreadsheet
sh = gc.open('Кириллово общий индекс')

births = sh.worksheet('Рождения')
birthsData = births.get_all_values()
births_df = pd.DataFrame(birthsData[1:], columns=birthsData[0])

marriages = sh.worksheet('Браки')
marriagesData = marriages.get_all_values()
marriages_df = pd.DataFrame(marriagesData[1:], columns=marriagesData[0])

deaths = sh.worksheet('Смерти')
deathsData = deaths.get_all_values()
deaths_df = pd.DataFrame(deathsData[1:], columns=deathsData[0])

people = sh.worksheet('Люди')
peopleData = people.get_all_values()
people_df = pd.DataFrame(peopleData[1:], columns=peopleData[0])

# Шаг 2

Подготовка вспомогательных dataframes.
- Сбор всех имен в общий список
- Нормализация имен для поиска дубликатов
- Обработка подсказок

# Шаг 3

Поиск упоминаний людей в разных метриках. Оценка вероятности совпадения.

In [18]:
# Configure pandas


# Set display options to avoid word wrap
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)        # Set width to a large value

births_columns_to_drop = ['Имя в документе','Отец в документе','Мать в документе','Воспреемник 1 в документе','Воспреемник 2 в документе']
marriages_columns_to_drop = []
deaths_columns_to_drop = ['Имя в документе','Имя родственника в документе']

In [45]:
# 100% упоминания

def find_person_by_id(name_id):
  mentions1 = births_df[
      (births_df['Имя'] == full_name) |
      (births_df['Отец'] == full_name) |
      (births_df['Мать'] == full_name) |
      (births_df['Воспреемник 1'] == full_name) |
      (births_df['Воспреемник 2'] == full_name)
  ]

  mentions2 = marriages_df[
      (marriages_df['Имя жениха'] == full_name) |
      (marriages_df['Имя невесты'] == full_name) |
      (marriages_df['Поручитель 1'] == full_name) |
      (marriages_df['Поручитель 2'] == full_name) |
      (marriages_df['Поручитель 3'] == full_name) |
      (marriages_df['Поручитель 4'] == full_name)
  ]

  mentions3 = deaths_df[
      (deaths_df['Имя'] == full_name) |
      (deaths_df['Имя родственника'] == full_name)
  ]

  # Function to print DataFrame or a custom message if empty
  def print_dataframe(df, title):
      if df.empty:
          print(f"{title}: No data available.")
      else:
          print(title)
          print(df)

  # Print each DataFrame with a custom message for empty DataFrames
  print_dataframe(mentions1.drop(columns=births_columns_to_drop), "Births DataFrame")
  print('-')
  print_dataframe(mentions2.drop(columns=marriages_columns_to_drop), "Marriages DataFrame")
  print('-')
  print_dataframe(mentions3.drop(columns=deaths_columns_to_drop), "Deaths DataFrame")
  print('-')

  return mentions1, mentions2, mentions3


# person = people_df.iloc[0]
# full_name = person['Полное имя c номером']
full_name = 'Иван Павлов [1482]'
birth_mentions, mentions2, mentions3 = find_person_by_id(full_name)

def estimate_parent_age(child_birth_date):
  # Subtract 16 years from the birth date to get the minimum estimate.
  estimated_birth_date_high = child_birth_date - pd.DateOffset(years=16)

  # Use the estimated maximum age of 50 years for the father.
  estimated_birth_date_low = child_birth_date - pd.DateOffset(years=50)
  return estimated_birth_date_low, estimated_birth_date_high

def estimate_godparent_age(child_birth_date):
  # Subtract 13 years from the birth date to get the minimum estimate.
  estimated_birth_date_high = child_birth_date - pd.DateOffset(years=16)

  # Use the estimated maximum age of 60 years for the god father.
  estimated_birth_date_low = child_birth_date - pd.DateOffset(years=50)
  return estimated_birth_date_low, estimated_birth_date_high

def estimate_birth_date(full_name, birth_mentions, mentions2, mentions3):
  # Check if the birth record exists
  if not birth_mentions.empty:
    # Check if there is a record for the person
    birth_record = birth_mentions[birth_mentions['Имя'] == full_name]
    if not birth_record.empty:
      birth_date = pd.to_datetime(birth_record['Дата'], format='%d.%m.%Y').iloc[0]
      print(f"Birth Date: {birth_date.strftime('%d-%m-%Y')}")
      return None,birth_date

    # Check if the person was a father or a mother
    father_record = birth_mentions[birth_mentions['Отец'] == full_name]
    if not father_record.empty:
      child_birth_date = pd.to_datetime(father_record['Дата'], format='%d.%m.%Y').iloc[0]
      return estimate_parent_age(child_birth_date)
    else:
      mother_record = birth_mentions[birth_mentions['Мать'] == full_name]
      if not mother_record.empty:
        child_birth_date = pd.to_datetime(mother_record['Дата'], format='%d.%m.%Y').iloc[0]
        return estimate_parent_age(child_birth_date)

    # Check if the person was one of god parents
    godparent1_record = birth_mentions[birth_mentions['Воспреемник 1'] == full_name]
    if not godparent1_record.empty:
      child_birth_date = pd.to_datetime(godparent1_record['Дата'], format='%d.%m.%Y').iloc[0]
      return estimate_godparent_age(child_birth_date)
    else:
      godparent2_record = birth_mentions[birth_mentions['Воспреемник 2'] == full_name]
      if not godparent2_record.empty:
        child_birth_date = pd.to_datetime(godparent2_record['Дата'], format='%d.%m.%Y').iloc[0]
        return estimate_godparent_age(child_birth_date)
  else:
    print("Birth record not found.")

estimated_birth_date_low, estimated_birth_date_high = estimate_birth_date(full_name, birth_mentions, mentions2, mentions3)
print(f"Estimated Birth Date (Min): {estimated_birth_date_low.strftime('%d-%m-%Y')} (Max): {estimated_birth_date_high.strftime('%d-%m-%Y')}")

Births DataFrame
    Номер муж Номер жен        Дата           Место                     Имя                 Отец Мать       Воспреемник 1 Воспреемник 2
267         6            19.01.1823  д. Гоголев Бор  Тимофей Федоров [1399]  Федор Петров [1442]       Иван Павлов [1482]              
-
Marriages DataFrame: No data available.
-
Deaths DataFrame: No data available.
-
Estimated Birth Date (Min): 19-01-1773 (Max): 19-01-1807


In [11]:
# Possible mentions

# person = people_df.iloc[0]
# full_name = person['Полное имя']
full_name = 'Петр Иванов'

mentions1 = births_df[
    (births_df['Имя'].str.contains(full_name, case=False, na=False)) |
    (births_df['Отец'].str.contains(full_name, case=False, na=False)) |
    (births_df['Мать'].str.contains(full_name, case=False, na=False)) |
    (births_df['Воспреемник 1'].str.contains(full_name, case=False, na=False)) |
    (births_df['Воспреемник 2'].str.contains(full_name, case=False, na=False))
]

mentions2 = marriages_df[
    (marriages_df['Имя жениха'].str.contains(full_name, case=False, na=False)) |
    (marriages_df['Имя невесты'].str.contains(full_name, case=False, na=False)) |
    (marriages_df['Поручитель 1'].str.contains(full_name, case=False, na=False)) |
    (marriages_df['Поручитель 2'].str.contains(full_name, case=False, na=False)) |
    (marriages_df['Поручитель 3'].str.contains(full_name, case=False, na=False)) |
    (marriages_df['Поручитель 4'].str.contains(full_name, case=False, na=False))
]

mentions3 = deaths_df[
    (deaths_df['Имя'].str.contains(full_name, case=False, na=False)) |
    (deaths_df['Имя родственника'].str.contains(full_name, case=False, na=False))
]

# Function to print DataFrame or a custom message if empty
def print_dataframe(df, title):
    if df.empty:
        print(f"{title}: No data available.")
    else:
        print(title)
        print(df)

# Print each DataFrame with a custom message for empty DataFrames
print_dataframe(mentions1.drop(columns=births_columns_to_drop), "Births DataFrame")
print('-')
print_dataframe(mentions2.drop(columns=marriages_columns_to_drop), "Marriages DataFrame")
print('-')
print_dataframe(mentions3.drop(columns=deaths_columns_to_drop), "Deaths DataFrame")


Births DataFrame
    Номер муж Номер жен        Дата           Место                    Имя                 Отец Мать                   Воспреемник 1           Воспреемник 2
13         10            15.01.1822  д. Гоголев Бор       Петр Иванов [17]  Иван Тимофеев [247]       Тимофей Федоров -дьячек- [507]   Ксения Семенова [522]
132        84            30.07.1822     с. Кирилово    Степан Петров [134]    Петр Иванов [151]            Параскева Антонова [1163]                        
149        98            24.08.1822     с. Кирилово      Петр Иванов [222]   Иван Семенов [381]             Никифор Спиридонов [984]  Евдокия Петрова [1194]
220       144            26.11.1822     с. Кирилово      Петр Иванов [364]    Иван Казмин [453]                 Фрол Гаврилов [1277]  Татьяна Казмина [1318]
277        14            11.02.1823    с. Кирилово   Василий Петров [1060]   Петр Иванов [1451]                Павел Васильев [1479]                        
-
Marriages DataFrame: No data available.

# Шаг 4

Вывод древа

In [ ]:
import graphviz
from IPython.display import Image

# Visualization using Graphviz
def visualize_family_tree():
  dot = graphviz.Digraph(comment='Family Tree')
  dot.attr('node', shape='box')

  for index, row in births_df.iterrows():

    # row=births_df.iloc[0]

    person = row['Имя']
    if person == '':
      continue
    dot.node(person, f'{person}\n{row["Дата"]}')
    father = row['Отец']
    dot.node(father)
    dot.edge(father, person, color='blue')
    mother = row['Мать']
    if mother != '':
      dot.node(mother)
      dot.edge(mother, person)
    godparent1 = row['Воспреемник 1']
    if godparent1 != '':
      dot.node(godparent1)
      dot.edge(godparent1, person, style='dashed')
    godparent2 = row['Воспреемник 2']
    if godparent2 != '':
      dot.node(godparent2)
      dot.edge(godparent2, person, style='dashed')

  dot = dot.unflatten(stagger=5)
  return dot

visualize_family_tree()